In [2]:
import arcade

GRAVITY = 0.8
SCREEN_WIDTH = 1280
SCREEN_HEIGHT = 720
SCREEN_TITLE = "Back2Mars"

CHARACTER_SCALING = 0.20
ENEMY_SCALING = 0.70
TILE_SCALING = 0.5
LAVA_SCALING = 0.7
SPIKE_SCALING = 0.4
UFO_SCALING = 0.75
PLAYER_MOVEMENT_SPEED = 5


class MenuView(arcade.View):
    def __init__(self):
        super().__init__()
        self.setup()

    def setup(self):
        self.background = arcade.load_texture("Space.png")

    def on_draw(self):
        self.clear()
        arcade.draw_texture_rect(
            self.background,
            arcade.LBWH(0, 0, SCREEN_WIDTH, SCREEN_HEIGHT),
        )

        arcade.draw_text(
            "Back 2 Mars",
            SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
            arcade.color.WHITE, font_size=50, anchor_x="center"
        )

        arcade.draw_text(
            "Click to continue",
            SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 75,
            arcade.color.WHITE, font_size=20, anchor_x="center"
        )

    def on_mouse_press(self, _x, _y, _button, _modifiers):
        instructions_view = InstructionView()
        self.window.show_view(instructions_view)


class InstructionView(arcade.View):
    def on_show_view(self):
        self.window.background_color = arcade.color.BLACK

    def on_draw(self):
        self.clear()
        arcade.draw_text(
            "MISSION: Help the Alien Escape the Facility!",
            SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
            arcade.color.WHITE, font_size=40, anchor_x="center"
        )
        arcade.draw_text(
            "Click to continue",
            SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 75,
            arcade.color.WHITE, font_size=20, anchor_x="center"
        )

    def on_mouse_press(self, _x, _y, _button, _modifiers):
        game_view = GameView()
        game_view.setup_level_1()
        self.window.show_view(game_view)


class GameOverView(arcade.View):
    def on_draw(self):
        self.clear()
        arcade.draw_text("You were captured and returned to your cell :(! ", SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
                         arcade.color.WHITE, 30, anchor_x="center")

        arcade.draw_text("Click to restart",
                         SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 75,
                         arcade.color.WHITE, 20, anchor_x="center")

    def on_mouse_press(self, x, y, button, modifiers):
        game = GameView()
        game.setup_level_1()
        self.window.show_view(game)

#Level 4
class VictoryLevelView(arcade.View):
    def __init__(self):
        super().__init__()
        self.background = arcade.load_texture("marsparty.jpg")
        self.alien = None
        self.phys = None
        self.jump_timer = 0
        self.alien_list = None

    def setup(self):
        self.alien = arcade.Sprite("alien.png", CHARACTER_SCALING)
        self.alien.center_x = SCREEN_WIDTH / 2
        self.alien.center_y = 150

        # SpriteList for safe drawing
        self.alien_list = arcade.SpriteList()
        self.alien_list.append(self.alien)

        ground = arcade.SpriteList(use_spatial_hash=True)
        for x in range(0, SCREEN_WIDTH + 60, 60):
            tile = arcade.Sprite(":resources:/images/tiles/planetMid.png", TILE_SCALING)
            tile.center_x = x
            tile.center_y = 40
            ground.append(tile)

        self.ground = ground

        self.phys = arcade.PhysicsEnginePlatformer(
            self.alien,
            self.ground,
            gravity_constant=GRAVITY
        )

    def on_draw(self):
        self.clear()
        arcade.draw_texture_rect(
            self.background,
            arcade.LBWH(0, 0, SCREEN_WIDTH, SCREEN_HEIGHT),
        )

        self.ground.draw()
        self.alien_list.draw()

        arcade.draw_text(
            "YOU MADE IT HOME!!! THE END",
            SCREEN_WIDTH / 2, 600,
            arcade.color.WHITE, 50, anchor_x="center"
        )

    def on_update(self, dt):
        self.phys.update()

        self.jump_timer += dt
        if self.jump_timer >= 1.0:
            self.alien.change_y = 18
            self.jump_timer = 0


#Level 3
class Level3View(arcade.View):
    def __init__(self):
        super().__init__()
        self.UFO = None
        self.scene = None
        self.alien_sprite = None
        self.alien_physics = None

        self.background = arcade.load_texture("outside.png")
        self.current_level = 3

    def setup_level_1(self):
        self.setup()

    def setup(self):
        self.scene = arcade.Scene()
        self.scene.add_sprite_list("Alien")
        self.scene.add_sprite_list("UFO")
        self.scene.add_sprite_list("Lavas")
        self.scene.add_sprite_list("Slab")

        self.alien_sprite = arcade.Sprite("alien.png", CHARACTER_SCALING)
        self.alien_sprite.center_x = 64
        self.alien_sprite.center_y = 180
        self.scene.add_sprite("Alien", self.alien_sprite)

        self.UFO = arcade.Sprite("UFO.png", UFO_SCALING)
        self.UFO.center_x = 1210
        self.UFO.center_y = 420
        self.scene.add_sprite("UFO", self.UFO)
        
        for x in range(0, 1300, 60):
            lava = arcade.Sprite(":resources:/images/tiles/lavaTop_high.png", TILE_SCALING)
            lava.center_x = x
            lava.center_y = 34
            self.scene.add_sprite("Lavas", lava)

        start_slab = arcade.Sprite(":resources:/images/tiles/stoneHalf_mid.png", TILE_SCALING)
        start_slab.center_x = 64
        start_slab.center_y = 128
        self.scene.add_sprite("Slab", start_slab)

        slab_positions = [
            [250, 150], [270, 150], 
            [450, 200],[500, 200],
            [690, 250], [750, 250], [745, 250], [800, 250],
            [945, 350], [1000, 350], [1190, 350], [1250, 350]
        ]

        for pos in slab_positions:
            slab = arcade.Sprite(":resources:/images/tiles/stoneHalf_mid.png", TILE_SCALING)
            slab.position = pos
            self.scene.add_sprite("Slab", slab)

        self.alien_physics = arcade.PhysicsEnginePlatformer(
            self.alien_sprite,
            platforms=self.scene["Slab"],
            gravity_constant=GRAVITY
        )

    def on_draw(self):
        self.clear()
        arcade.draw_texture_rect(self.background, arcade.LBWH(0, 0, SCREEN_WIDTH, SCREEN_HEIGHT))
        self.scene.draw()

    def on_key_press(self, key, modifiers):
        if key == arcade.key.LEFT:
            self.alien_sprite.change_x = -PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.RIGHT:
            self.alien_sprite.change_x = PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.UP and self.alien_physics.can_jump():
            self.alien_sprite.change_y = 15

    def on_key_release(self, key, modifiers):
        if key in (arcade.key.LEFT, arcade.key.RIGHT):
            self.alien_sprite.change_x = 0

    def on_update(self, delta_time):
        self.alien_physics.update()

        if arcade.check_for_collision_with_list(self.alien_sprite, self.scene["Lavas"]):
            self.window.show_view(GameOverView())

        if self.alien_sprite.center_x > SCREEN_WIDTH - 10:
            level4 = VictoryLevelView()
            level4.setup()
            self.window.show_view(level4)

        if self.alien_sprite.center_x < 10:
            self.alien_sprite.center_x = 10

#level 2       
class Level2View(arcade.View):
    def __init__(self):
        super().__init__()
        self.user_text = ""

    def on_show_view(self):
        self.window.background_color = arcade.color.BLACK

    def on_draw(self):
        self.clear()

        arcade.draw_text("You escaped Agent Octo's lab.",
                         SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 100,
                         arcade.color.WHITE, 30, anchor_x="center")

        arcade.draw_text("To leave, guess a number from 1–10.",
                         SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50,
                         arcade.color.WHITE, 25, anchor_x="center")

        arcade.draw_text("Hint: How many hearts does an octopus have?",
                         SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2,
                         arcade.color.WHITE, 20, anchor_x="center")

        arcade.draw_text(f"Your answer: {self.user_text}",
                         SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 80,
                         arcade.color.LIGHT_GRAY, 25, anchor_x="center")

        arcade.draw_text("Press ENTER to submit",
                         SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 150,
                         arcade.color.GRAY, 20, anchor_x="center")

    def on_text(self, text):
        self.user_text += text

    def on_key_press(self, key, modifiers):
        if key == arcade.key.BACKSPACE:
            self.user_text = self.user_text[:-1]

        if key == arcade.key.ENTER:
            ans = self.user_text.strip().lower()
            if ans in ["3", "three"]:
                level3 = Level3View()
                level3.setup_level_1()
                self.window.show_view(level3)
            else:
                self.window.show_view(GameOverView())


#Level 1
class GameView(arcade.View):
    def __init__(self):
        super().__init__()

        self.scene = None
        self.alien_sprite = None
        self.man_sprite = None

        self.alien_physics = None
        self.man_physics = None

        self.background = arcade.load_texture("background_lab1.png")
        self.current_level = 1

    def setup_level_1(self):
        self.setup()

    def next_level(self):
        self.window.show_view(Level2View())

    def setup(self):
        self.scene = arcade.Scene()

        
        self.alien_sprite = arcade.Sprite("alien.png", CHARACTER_SCALING)
        self.alien_sprite.center_x = 64
        self.alien_sprite.center_y = 128
        self.scene.add_sprite("Alien", self.alien_sprite)

        
        self.man_sprite = arcade.Sprite("man.png", ENEMY_SCALING)
        self.man_sprite.center_x = 1250
        self.man_sprite.center_y = 100
        self.scene.add_sprite("Man", self.man_sprite)

        for x in range(0, 1300, 60):
            wall = arcade.Sprite(":resources:/images/tiles/stoneCenter.png", TILE_SCALING)
            wall.center_x = x
            wall.center_y = 34
            self.scene.add_sprite("Walls", wall)

        slab_positions = [
            [200, 150], [260, 150], [300, 150], [450, 200], [510, 200],
            [665, 250], [700, 250], [745, 250], [800, 250]
        ]
        for pos in slab_positions:
            slab = arcade.Sprite(":resources:/images/tiles/stoneHalf_mid.png", TILE_SCALING)
            slab.position = pos
            self.scene.add_sprite("Slab", slab)

        for pos in [[260, 207], [515, 257], [750, 308]]:
            spike = arcade.Sprite(":resources:/images/tiles/spikes.png", SPIKE_SCALING)
            spike.position = pos
            self.scene.add_sprite("Hazard", spike)

        for pos in [[180, 35], [600, 35], [802, 35], [891, 35], [980, 35]]:
            lava = arcade.Sprite(":resources:/images/tiles/lavaTop_high.png", LAVA_SCALING)
            lava.position = pos
            self.scene.add_sprite("Hazard", lava)

        self.alien_physics = arcade.PhysicsEnginePlatformer(
            self.alien_sprite,
            [self.scene["Walls"], self.scene["Slab"]],
            gravity_constant=GRAVITY
        )

        self.man_physics = arcade.PhysicsEnginePlatformer(
            self.man_sprite,
            platforms=self.scene["Walls"],
            gravity_constant=GRAVITY
        )

    def on_draw(self):
        self.clear()
        arcade.draw_texture_rect(self.background, arcade.LBWH(0, 0, SCREEN_WIDTH, SCREEN_HEIGHT))
        self.scene.draw()

    def on_key_press(self, key, modifiers):
        if key == arcade.key.LEFT:
            self.alien_sprite.change_x = -PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.RIGHT:
            self.alien_sprite.change_x = PLAYER_MOVEMENT_SPEED
        elif key == arcade.key.UP and self.alien_physics.can_jump():
            self.alien_sprite.change_y = 15

    def on_key_release(self, key, modifiers):
        if key in (arcade.key.LEFT, arcade.key.RIGHT):
            self.alien_sprite.change_x = 0

    def on_update(self, delta_time):
        self.alien_physics.update()
        self.man_physics.update()

        if self.man_sprite.center_x < self.alien_sprite.center_x:
            self.man_sprite.change_x = 3.25
        else:
            self.man_sprite.change_x = -3.25

        if arcade.check_for_collision(self.man_sprite, self.alien_sprite):
            self.window.show_view(GameOverView())

        if arcade.check_for_collision_with_list(self.alien_sprite, self.scene["Hazard"]):
            self.window.show_view(GameOverView())

        if self.alien_sprite.center_x > SCREEN_WIDTH - 10:
            self.next_level()

        if self.alien_sprite.center_x < 10:
            self.alien_sprite.center_x = 10


def main():
    window = arcade.Window(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
    window.show_view(MenuView())
    arcade.run()


if __name__ == "__main__":
    main()
